In [2]:
!pip install -q optuna


In [3]:
!pip install  -q monai


In [4]:
!pip install -q albumentations

In [5]:
!pip install -q git+https://github.com/huggingface/transformers.git

In [6]:
import os,sys
import numpy as np 
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from PIL import Image 
from tqdm.notebook import tqdm
from IPython.display import clear_output

In [7]:
#all required torch libraries
import torch
import torchvision
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms,utils
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torchvision.transforms import ToPILImage
from torch.utils.data.dataset import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision.ops import masks_to_boxes
from albumentations import Compose, Transpose, VerticalFlip, HorizontalFlip, RandomRotate90, ShiftScaleRotate, OpticalDistortion

In [8]:

import time
import optuna
import csv
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("finished")
from os import listdir
from os.path import isfile, join

finished


In [9]:
import monai

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [10]:
def resize(path):
  dirs = os.listdir( path )
  for item in tqdm(dirs):
    if os.path.isfile(path+item):
      im = Image.open(path+item)
      f, e = os.path.splitext(path+item)
      imResize = im.resize((256,256), Image.NEAREST)
      imResize.save(f+e, 'PNG', quality=100)
      
label_path =  "/kaggle/input/nucleus-data-c-elegans/nucleus_data/segmentation_maps"
output_features_path = "/kaggle/input/nucleus-data-c-elegans/nucleus_data/features"
resize(label_path)

  0%|          | 0/6790 [00:00<?, ?it/s]

In [11]:
ids=[]
label_filenames = [f for f in listdir(label_path) if isfile(join(label_path, f))]
feature_filenames = [f for f in listdir(output_features_path) if isfile(join(output_features_path, f))]
for i in range(len(feature_filenames)):
  ids.append(feature_filenames[i][1:])
print(len(ids))

df = pd.DataFrame(ids ,columns=["file_ids"])
df.to_csv('full_file_ids.csv', index=False)

#sanity check
df = pd.read_csv('full_file_ids.csv')
df.head()

6756


,file_ids
0,182_22.png
1,167_27.png
2,86_29.png
3,154_16.png
4,177_8.png


In [12]:
import pandas as pd
import os
import cv2

# Read the CSV file
df = pd.read_csv('full_file_ids.csv')

# Extract the file_ids column as a list
ids = df['file_ids'].tolist()

# Filter non-empty file IDs
non_empty_ids = []


for file_id in ids:
    mask_path = os.path.join(label_path, 'L' + file_id)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if cv2.countNonZero(mask) > 0:
        non_empty_ids.append(file_id)

# Create a new DataFrame with non-empty file IDs
df_non_empty = pd.DataFrame(non_empty_ids, columns=["file_ids"])

# Save the non-empty file IDs to a new CSV file
df_non_empty.to_csv('file_ids.csv', index=False)

# Read the file_ids CSV file
df = pd.read_csv('file_ids.csv')

# Sort the file_ids column
df.sort_values(by='file_ids', inplace=True)

# Display the first few rows of the sorted DataFrame
print(df.head())


      file_ids
2625  0_10.png
1544  0_11.png
2409  0_12.png
1479  0_13.png
2987  0_14.png


In [13]:
pip install 'transformers[torch]'

Note: you may need to restart the kernel to use updated packages.


In [14]:
class SegmentationDataset(Dataset):
    def __init__(self, csv, processor):
        self.df = pd.read_csv(csv)
        self.ids = self.df["file_ids"]
        self.processor = processor
        

    def __getitem__(self, idx):
        image = Image.open("/kaggle/input/nucleus-data-c-elegans/nucleus_data/features/F" + self.ids[idx])
        mask = np.array(Image.open("/kaggle/input/nucleus-data-c-elegans/nucleus_data/segmentation_maps/L" + self.ids[idx]))
            

        prompt = self.mask_to_boxes(mask)

        inputs = self.processor(image, input_boxes=[prompt], return_tensors="pt")
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}

        inputs["ground_truth_mask"] = mask

        return inputs

    def __len__(self):
        return len(self.ids)
    
    
    @staticmethod
    def mask_to_boxes(mask):
        obj_id=np.unique(mask)
        obj_ids=obj_id[1:]
        maskes = mask == obj_ids[:, None, None]
        b_boxes=[]
        num_objs=len(obj_ids)
        for i in range(num_objs):
            pos=np.nonzero(maskes[i])
            xmin=np.min(pos[1])
            xmax=np.max(pos[1])
            ymin=np.min(pos[0])
            ymax=np.max(pos[0])
            b_boxes.append([xmin,ymin,xmax,ymax])
        
        return b_boxes
            
            

In [15]:
from transformers import SamProcessor

processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

In [16]:
train_dataset = SegmentationDataset(csv = "file_ids.csv", processor=processor)

In [17]:
example = train_dataset[4235]
for k,v in example.items():
  print(k,v.shape)

pixel_values torch.Size([3, 1024, 1024])
original_sizes torch.Size([2])
reshaped_input_sizes torch.Size([2])
input_boxes torch.Size([20, 4])
ground_truth_mask (256, 256)


In [18]:
import torch

def collate_fn(batch):
    images = []
    masks = []
    boxes = []

    for item in batch:
        images.append(item["pixel_values"])
        masks.append(item["ground_truth_mask"])
        boxes.append(item["input_boxes"])

    
    images = torch.stack(images, dim=0)
    masks = torch.stack([torch.from_numpy(mask) for mask in masks], dim=0)

    
    targets = []
    max_num_boxes = max(len(b) for b in boxes)
    for b in boxes:
        padded_boxes = torch.cat([b, torch.zeros((max_num_boxes - len(b), 4))], dim=0)
        targets.append(padded_boxes)

    
    targets = torch.stack(targets, dim=0)

    # Create a dictionary to hold the input data
    inputs = {
        "pixel_values": images,
        "ground_truth_mask": masks,
        "input_boxes": targets
    }

    return inputs


In [19]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True,collate_fn=collate_fn)
    

In [21]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
  print(k,v.shape)

pixel_values torch.Size([2, 3, 1024, 1024])
ground_truth_mask torch.Size([2, 256, 256])
input_boxes torch.Size([2, 86, 4])
